# Part 2 - Implementation 

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=1198865 sha256=5b9f6bf4bedffa30b0db5a590b3cace2f0f910df2117328b43f42489d6c44eca
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
import pandas as  pd
import numpy as np

from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split as TTS1
from surprise import KNNWithMeans

from sklearn.model_selection import train_test_split as TTS2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'surprise'

In [4]:
mov_df = pd.read_csv('movie_genres.csv').drop("Unnamed: 0",axis=1)
mov_df['movie_id']= pd.factorize(mov_df['movie_title'])[0]

rev_df = pd.read_csv('user_reviews.csv')
rev_df['user_id']= pd.factorize(rev_df['User'])[0]

df_melt = rev_df.melt(id_vars=['user_id'], value_vars=rev_df.columns.difference(['User','user_id']))
df_melt.rename(columns = {'variable':'movie_title'}, inplace=True)

df_merge = pd.merge(df_melt, mov_df, on='movie_title').drop('movie_title',axis=1)
df_merge.insert(1, 'movie_id', df_merge.pop('movie_id'))

In [5]:
print(df_merge)

         user_id  movie_id  value  genre_action  genre_adventure  \
0              0       440    0.0             0                0   
1              1       440    0.0             0                0   
2              2       440    0.0             0                0   
3              3       440    0.0             0                0   
4              4       440    0.0             0                0   
...          ...       ...    ...           ...              ...   
1199995      595      1922    0.0             1                0   
1199996      596      1922    0.0             1                0   
1199997      597      1922    0.0             1                0   
1199998      598      1922    0.0             1                0   
1199999      599      1922    0.0             1                0   

         genre_animation  genre_biography  genre_comedy  genre_crime  \
0                      0                0             0            1   
1                      0               

In [5]:
##Find unique movies
df_merge['movie_id'].nunique()

2000

In [9]:
##Number of reviews
##top movies 20, lowest 1.

df_merge.loc[df_merge['value']>0]['movie_id'].value_counts()

143     20
1894    20
1178    20
1944    19
1666    19
        ..
1304     1
958      1
774      1
604      1
388      1
Name: movie_id, Length: 2000, dtype: int64

In [67]:
#Dataframe with number of reviews
rev_count= pd.DataFrame(df_merge.loc[df_merge['value']>0]['movie_id'].value_counts())

#Find the movies with less than 5 reviews
rare_movies = rev_count[rev_count['movie_id'] < 5].index

#Get the movies with 10 or more reviews
common_movies = df_merge[~df_merge['movie_id'].isin(rare_movies)]

common_movies.shape


(1079400, 28)

In [56]:
# Unique movies with more than 1000 reviews:
common_movies['movie_id'].nunique()

651

Test KNN on cleaned data

In [24]:
data_pred = common_movies.loc[common_movies['value']==0]
X_pred = data_pred.drop('value', axis=1)
Y_pred = data_pred['value']

data_rated = common_movies.loc[common_movies['value']>0]

X = data_rated.drop('value', axis=1)
Y = data_rated['value']

Xtrain, Xtest, Ytrain, Ytest = TTS2(X,Y, test_size= .2, random_state=42)

clf = KNeighborsClassifier(n_neighbors=500)
clf.fit(Xtrain,Ytrain)

pred = clf.predict(Xtest)
exact_acc = accuracy_score(Ytest,pred)

bin_rev_real = np.where(Ytest >= 3,1,0)
bin_rev_pred = np.where(pred >=3,1,0)

bin_acc =  sum(a == b for a, b in zip(bin_rev_real, bin_rev_pred)) / len(bin_rev_real)

print(exact_acc)
print(bin_acc)

0.2736773350751143
0.8040496407576747


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=430916cd-0b84-4120-a50c-61096e15b16a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>